<a href="https://colab.research.google.com/github/thislis/KSHS/blob/main/%EB%AC%B4%EC%B9%9C%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

딥러닝을 이용한 이미지 처리 - 글씨 인식(OCR)  
목표 1. 글씨 이미지를 분석해 어느 글꼴에 가까운지 판단  
목표 2. 자신의 글씨체와 가장 가까운 글꼴로 이미지를 텍스트로 변환

In [ ]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!nvidia-smi -l 1

In [ ]:
!pip install kora
from kora import console
console.start()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



In [ ]:
#필요한 모듈 임포트

import re, os
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
data_path = "/content/drive/Shareddrives/횡성 영재/data"

walrdeu = os.listdir(data_path + '/NanumWaIrDeu.ttf')
oeharmeonigeulssi = os.listdir(data_path + '/NanumOeHarMeoNiGeurSsi.ttf')
himnaeraneunmarbodan = os.listdir(data_path + '/NanumHimNaeRaNeunMarBoDan.ttf')
harabeojieuinanum = os.listdir(data_path + '/NanumHarABeoJiEuiNaNum.ttf')
yageunhaneungimjuim = os.listdir(data_path + '/NanumYaGeunHaNeunGimJuIm.ttf')

In [ ]:
TRAINING_DIR = "/content/drive/Shareddrives/횡성 영재/data"
VALIDATION_DIR = "/content/drive/Shareddrives/횡성 영재/test"
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(128,128),
class_mode='categorical',
batch_size=128
)


validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(128,128),
class_mode='categorical',
batch_size=128
)

print(train_generator)

model = tf.keras.models.Sequential([
  # Note the input shape is the desired size of the image 128** with 3 bytes color
  # This is the first convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  # The second convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The third convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The fourth convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  # 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])


model.summary()

model_dir = '/content/drive/Shareddrives/횡성 영재/model'
model_path = model_dir + "/classify.model"
# 모델결과 저장 및 earlystopping
checkpoint = ModelCheckpoint(filepath = model_path, monitor='val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 6)


model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, batch_size = 128, epochs=30, validation_data = validation_generator, callbacks = [checkpoint, early_stopping])

Found 55860 images belonging to 5 classes.
Found 30543 images belonging to 5 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                    

437/437 [==============================] - 329s 741ms/step - loss: 0.7557 - accuracy: 0.6722 - val_loss: 0.4924 - val_accuracy: 0.7216
Epoch 2/30
437/437 [==============================] - ETA: 0s - loss: 0.2612 - accuracy: 0.9012
Epoch 2: val_loss improved from 0.49236 to 0.29610, saving model to /content/drive/Shareddrives/횡성 영재/model/cloud_classify.model


437/437 [==============================] - 299s 683ms/step - loss: 0.2612 - accuracy: 0.9012 - val_loss: 0.2961 - val_accuracy: 0.8988
Epoch 3/30
437/437 [==============================] - ETA: 0s - loss: 0.1455 - accuracy: 0.9531
Epoch 3: val_loss did not improve from 0.29610
437/437 [==============================] - 296s 678ms/step - loss: 0.1455 - accuracy: 0.9531 - val_loss: 0.4319 - val_accuracy: 0.8525
Epoch 4/30
437/437 [==============================] - ETA: 0s - loss: 0.1045 - accuracy: 0.9691
Epoch 4: val_loss improved from 0.29610 to 0.22822, saving model to /content/drive/Shareddrives/횡성 영재/model/cloud_classify.model


437/437 [==============================] - 297s 679ms/step - loss: 0.1045 - accuracy: 0.9691 - val_loss: 0.2282 - val_accuracy: 0.9151
Epoch 5/30
437/437 [==============================] - ETA: 0s - loss: 0.0861 - accuracy: 0.9759
Epoch 5: val_loss improved from 0.22822 to 0.14756, saving model to /content/drive/Shareddrives/횡성 영재/model/cloud_classify.model


437/437 [==============================] - 298s 683ms/step - loss: 0.0861 - accuracy: 0.9759 - val_loss: 0.1476 - val_accuracy: 0.9319
Epoch 6/30
437/437 [==============================] - ETA: 0s - loss: 0.0673 - accuracy: 0.9815
Epoch 6: val_loss did not improve from 0.14756
437/437 [==============================] - 296s 678ms/step - loss: 0.0673 - accuracy: 0.9815 - val_loss: 0.2792 - val_accuracy: 0.9134
Epoch 7/30
437/437 [==============================] - ETA: 0s - loss: 0.0620 - accuracy: 0.9820
Epoch 7: val_loss did not improve from 0.14756
437/437 [==============================] - 296s 678ms/step - loss: 0.0620 - accuracy: 0.9820 - val_loss: 1.8753 - val_accuracy: 0.5165
Epoch 8/30
437/437 [==============================] - ETA: 0s - loss: 0.0668 - accuracy: 0.9837
Epoch 8: val_loss did not improve from 0.14756
437/437 [==============================] - 297s 680ms/step - loss: 0.0668 - accuracy: 0.9837 - val_loss: 0.5608 - val_accuracy: 0.8722
Epoch 9/30
437/437 [===========